# FastAI
Using FastAI

In [ ]:
%cd /content/
!rm -rf /content/Whats-this-rock/
# !git clone https://github.com/udaylunawat/Whats-this-rock.git
!git clone -b nbdev https://github.com/udaylunawat/Whats-this-rock.git

In [ ]:
%cd /content/Whats-this-rock/

In [ ]:
!nvidia-smi

In [ ]:
!sh src/scripts/setup.sh

## Downloading data and moving bad, corrupted and duplicate images
### Downloading data

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!rm -rf data/bad_images/

In [ ]:
del get_data, process_data

In [ ]:
import logging

# STEP 1
# create a logger object instance
logger = logging.getLogger()

# STEP 2
# specifies the lowest severity for logging
logger.setLevel(logging.NOTSET)

# STEP 3
# set a destination for your logs or a "handler"
# here, we choose to print on console (a consoler handler)
console_handler = logging.StreamHandler()

# STEP 4
# set the logging format for your handler
log_format = '%(asctime)s | %(levelname)s: %(message)s'
console_handler.setFormatter(logging.Formatter(log_format))

# finally, we add the handler to the logger
logger.addHandler(console_handler)

# the second handler is a file handler
file_handler = logging.FileHandler('sample.log')
file_handler.setLevel(logging.INFO)
file_handler_format = '%(asctime)s | %(levelname)s | %(lineno)d: %(message)s'
file_handler.setFormatter(logging.Formatter(file_handler_format))
logger.addHandler(file_handler)

In [ ]:
import omegaconf
import subprocess
from src.data.download import get_data
from src.data.preprocess import process_data

cfg = omegaconf.OmegaConf.load('configs/config.yaml')
cfg.dataset_id = [1,2,3,4]

subprocess.run(["sh", "src/scripts/clean_dir.sh"], stdout=subprocess.PIPE).stdout.decode("utf-8")
get_data()
process_data(cfg)

### Remove duplicate images

In [ ]:
# https://github.com/saifjamsheer/google-dataset-creator/tree/c458ec1f2e676f474527cbf3ee2da3937d260d0c
from imutils import paths
from pathlib import Path
import os
import cv2
import argparse
from src.data.utils import timer_func, find_filepaths


"""
Script that deletes duplicate images from a dataset. This
can be used in conjunction with 'create.py' to delete
any duplicate images before manual pruning is done. 
"""
def dhash(image, hashSize=8):
    # Convert the image to grayscale and resize it
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (hashSize + 1, hashSize))
    # Compute the horizontal gradient between adjacent pixels
    diff = resized[:, 1:] > resized[:, :-1]
    # Converts the image to a hash and returns it
    hash = sum([2**i for (i,j) in enumerate(diff.flatten()) if j])
    return hash


@timer_func
def remove_duplicates(dir):
    # Path of the dataset of images
    output_path = dir
    hashes = dict()

    # List of paths of the images in the datasets
    image_paths = list(paths.list_images(output_path))

    for image_path in image_paths:
        # Load input image
        
        image = cv2.imread(image_path)
        # Compute the hash of the image
        try:
            h = dhash(image)
        except:
            print(f'cv2.imread {image_path} Returns None!')
        # Store all images in a dictionary of hashes
        path = hashes.get(h, [])
        path.append(image_path)
        hashes[h] = path

    # Iterate through the hashes
    for h, path_value in hashes.items():
        # Checking if a duplicate exists
        if len(path_value) > 1:
            # Deleting all duplicates
            for path in path_value[1:]:
                print("[INFO] Deleting: {}".format(path))
                # os.remove(path)
    return list({k: v for (k, v) in hashes.items() if len(v) > 1}.values())

In [ ]:
duplicates = remove_duplicates('data/2_processed')

In [ ]:
!pip install imagededup

In [ ]:
from imagededup.methods import PHash
import random

def find_duplicates(path):
    phasher = PHash()
    encodings = phasher.encode_images(image_dir=path)
    duplicates = phasher.find_duplicates(encoding_map=encodings)
    duplicates = {k: v for (k, v) in duplicates.items() if len(v) > 0}
    
    from imagededup.utils import plot_duplicates
    plot_duplicates(image_dir=path,
                    duplicate_map=duplicates,
                    filename=random.choice(list(duplicates.values()))[0]);

find_duplicates('data/2_processed/Coal')

### Removing bad images

In [ ]:
# Python program to store list to file using pickle module
import pickle

# write list to binary file
def write_list(names, filename='delfile'):
    # store list in binary file so 'wb' mode
    with open(filename, 'wb') as fp:
        pickle.dump(names, fp)
        print('Done writing list into a binary file')

# Read list to memory
def read_list(filename='delfile'):
    # for reading also binary mode is important
    with open(filename, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

# del_list = read_list()
# print('List is', del_list)

In [ ]:
bad_images = read_list('updated_delete_pickle')

In [ ]:
import os
import shutil
os.makedirs('data/bad_images', exist_ok=True)

for path in bad_images:
    if os.path.exists(path):
        shutil.move(str(path), 'data/bad_images')

In [ ]:
# assert len(os.listdir('data/bad_images')) == len(bad_images)

## Loading data

In [ ]:
from fastai.vision.all import *

In [ ]:
dls = ImageDataLoaders.from_folder(path='data/2_processed/', valid_pct=0.2,
                                   item_tfms=Resize(224))
dls.valid_ds.items[:3]

In [ ]:
print(f'Training set samples:- {len(dls.train_ds)} images.\nValidation set samples:- {len(dls.valid_ds)} images.')

In [ ]:
dls.show_batch()

In [ ]:
learn = vision_learner(dls, resnet34, metrics=[error_rate, F1Score(average='macro')])


In [ ]:
learn.fine_tune(5, cbs=ShowGraphCallback())

Lowest error rate:- 0.2460, val_loss:- 0.93

In [ ]:
# save model
learn.export(file='model.pkl')

learn = load_learner('model.pkl')

In [ ]:
learn.predict('data/4_tfds_dataset/test/Basalt/Basalt_104.jpg')

In [ ]:
def get_results(learn):
    print(f'\nSample Results:')
    learn.show_results()
    plt.show()

    interp = Interpretation.from_learner(learn)
    print(f'\nTop losses:-')
    interp.plot_top_losses(9, figsize=(15,10))
    plt.show()

    interp = ClassificationInterpretation.from_learner(learn)
    losses,idxs = interp.top_losses()
    len(dls.valid_ds)==len(losses)==len(idxs)

    print(f'\nConfusion Matrix:-')
    interp.plot_confusion_matrix(figsize=(7,7))
    plt.show()

    print(f'\nNormalized Confusion Matrix:-')
    interp.plot_confusion_matrix(figsize=(7,7), normalize=True)
    plt.show()


In [ ]:
get_results(learn)

## Data Cleaning

### Remove bad images

In [ ]:
files_to_delete = []

In [ ]:
from fastai.vision.widgets import *
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): files_to_delete.append(cleaner.fns[idx])

In [ ]:
def tfds_to_processed(filepath):
    base = '/'.join(str(filepath).split('/')[-2:]).replace('\')','')
    processed = os.path.join('data/2_processed', base)
    return Path(set(processed))

new_paths = list(map(lambda x:tfds_to_processed(x), files_to_delete))

In [ ]:
# update delfile
paths = read_list()
write_list(paths+new_paths, 'delete_file_pickle')

In [ ]:
latest_del = paths+new_paths

In [ ]:
try: 
    for idx in paths:
        idx.unlink()
except: pass

write_list(paths, filename='delfile')

In [ ]:
!rm -rf data/4_tfds_dataset

In [ ]:
import splitfolders
splitfolders.ratio(
            "data/2_processed",
            output="data/4_tfds_dataset",
            ratio=(0.70, 0.15, 0.15),
            seed=42,
            move=False,
        )

Training after data cleaning

In [ ]:
dls = ImageDataLoaders.from_folder(path='data/4_tfds_dataset/',
                                   train='train',
                                   valid='val',
                                   item_tfms=Resize(224))
dls.valid_ds.items[:3]

In [ ]:
print(f"Number of images after deletetion:- ")
print(f'Training set samples:- {len(dls.train_ds)} images.\nValidation set samples:- {len(dls.valid_ds)} images.')

In [ ]:
dls.show_batch()

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(5, cbs=ShowGraphCallback())

## Using fastai Data Augmentation

In [ ]:
dls = ImageDataLoaders.from_folder(path='data/4_tfds_dataset/', train='train',
                                   valid='val', item_tfms=Resize(460),
                                   batch_tfms=aug_transforms(size=224))
dls.valid_ds.items[:3]

In [ ]:
dls.show_batch()

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)

In [ ]:
learn.lr_find()

Using optimal lr

In [ ]:
learn.fine_tune(5, 3e-3)

In [ ]:
get_results(learn)